# 02 Make Network Version 01
Step 2 applies project cards, makes Python changes, and writes out a pickle of the version 01 network

In [1]:
import os
import sys
import yaml
from subprocess import Popen
import pickle

import pandas as pd
import geopandas as gpd
import numpy as np


from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import WranglerLogger
from network_wrangler import ProjectCard

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import metcouncil

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

### Remote I/O and Parameters

In [4]:
root_dir = os.path.join('C:\\', 'Users', 'david.ory', 'Documents', 'GitHub', 'client_met_council_network')
input_dir = os.path.join(root_dir, 'network_standard')
pickle_dir = input_dir
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
lasso_dir = os.path.join('C:\\', 'Users', 'david.ory', 'Documents', 'GitHub', 'lasso')

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-10-16 11:42:03, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso
2020-10-16 11:42:03, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso


### Read Version 00 Scenario

In [6]:
version_00_pickle_file_name = os.path.join(pickle_dir, 'working_scenario_00.pickle')
v_01_scenario = pickle.load(open(version_00_pickle_file_name, 'rb'))

### Apply Project Cards

In [7]:
project_card_list = []
card_01_file_name = os.path.join(card_dir, 'year_2018_corrections_assign_group_roadway_class.yml')
card_02_file_name = os.path.join(card_dir, 'year_2018_corrections_assign_group.yml')

card = ProjectCard.read(card_01_file_name, validate = False)
project_card_list.append(card)

card = ProjectCard.read(card_02_file_name, validate = False)
project_card_list.append(card)

In [8]:
for project_card in project_card_list:
    v_01_scenario.apply_project(project_card)

2020-10-16 11:42:36, INFO: Applying Correct Year 2018 Assignment Group and Roadway Class
2020-10-16 11:42:36, INFO: Applying Correct Year 2018 Assignment Group and Roadway Class
2020-10-16 11:42:36, INFO: Applying Correct Year 2018 Assignment Group and Roadway Class
2020-10-16 11:42:36, INFO: Applying Correct Year 2018 Assignment Group and Roadway Class
2020-10-16 11:42:36, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group and Roadway Class
2020-10-16 11:42:36, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group and Roadway Class
2020-10-16 11:42:36, INFO: Applying Correct Year 2018 Assignment Group
2020-10-16 11:42:36, INFO: Applying Correct Year 2018 Assignment Group
2020-10-16 11:42:36, INFO: Applying Correct Year 2018 Assignment Group
2020-10-16 11:42:36, INFO: Applying Correct Year 2018 Assignment Group
2020-10-16 11:42:36, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-10-16 11:42:36, INFO: 

C:\Users\david.ory\anaconda3\envs\lasso_x\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2020-10-16 11:42:46, INFO: Applying Correct Year 2018 Assignment Group
2020-10-16 11:42:46, INFO: Applying Correct Year 2018 Assignment Group
2020-10-16 11:42:46, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-10-16 11:42:46, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-10-16 11:42:55, INFO: Applying Correct Year 2018 Assignment Group
2020-10-16 11:42:55, INFO: Applying Correct Year 2018 Assignment Group
2020-10-16 11:42:55, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-10-16 11:42:55, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-10-16 11:42:55, INFO: Applying Correct Year 2018 Assignment Group
2020-10-16 11:42:55, INFO: Applying Correct Year 2018 Assignment Group
2020-10-16 11:42:55, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-10-16 11:42:55, INFO: Applying Project to Roadway Network: Correct Year 20

In [9]:
v_01_scenario.applied_projects

['Correct Year 2018 Assignment Group and Roadway Class',
 'Correct Year 2018 Assignment Group']

### Make Travel Model Networks (for manual fixes)

In [10]:
m_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_01_scenario.road_net, 
    parameters = parameters)

2020-10-16 11:47:10, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso
2020-10-16 11:47:10, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso


### Update Number of Lanes

In [11]:
m_net = metcouncil.calculate_number_of_lanes(
        roadway_net=m_net,
        parameters=parameters,
        overwrite=False,
)
m_net.links_df.lanes.value_counts()

2020-10-16 11:47:11, INFO: Calculating Number of Lanes as network variable: 'lanes'
2020-10-16 11:47:11, INFO: Calculating Number of Lanes as network variable: 'lanes'
2020-10-16 11:47:11, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso
2020-10-16 11:47:11, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso
2020-10-16 11:47:11, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2020-10-16 11:47:11, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2020-10-16 11:47:11, INFO: Finished calculating centroid connector variable: centroidconnect
2020-10-16 11:47:11, INFO: Finished calculating centroid connector variable: centroidconnect
2020-10-16 11:50:09, INFO: Finished calculating number of lanes to: lanes
2020-10-16 11:50:09, INFO: Finished calculating number of lanes to: lanes


1    325181
2     26560
3      1734
4       545
5        66
6         6
Name: lanes, dtype: int64

In [12]:
m_net.roadway_standard_to_met_council_network()

2020-10-16 11:50:09, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2020-10-16 11:50:09, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2020-10-16 11:50:09, INFO: Didn't detect managed lanes in network.
2020-10-16 11:50:09, INFO: Didn't detect managed lanes in network.
2020-10-16 11:50:09, INFO: Creating calculated roadway variables.
2020-10-16 11:50:09, INFO: Creating calculated roadway variables.
2020-10-16 11:50:09, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type
2020-10-16 11:50:09, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type


c:\users\david.ory\documents\github\lasso\lasso\roadway.py:427: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2020-10-16 11:51:51, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2020-10-16 11:51:51, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2020-10-16 11:51:53, INFO: Adding roadway network variable for county using a spatial join with: C:\Users\david.ory\Documents\GitHub\lasso\metcouncil_data\county\cb_2017_us_county_5m.shp
2020-10-16 11:51:53, INFO: Adding roadway network variable for county using a spatial join with: C:\Users\david.ory\Documents\GitHub\lasso\metcouncil_data\county\cb_2017_us_county_5m.shp


c:\users\david.ory\documents\github\lasso\lasso\roadway.py:292: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2020-10-16 11:53:08, INFO: Finished Calculating county variable: county
2020-10-16 11:53:08, INFO: Finished Calculating county variable: county
2020-10-16 11:53:09, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2020-10-16 11:53:09, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2020-10-16 11:53:09, INFO: Calculating MPO as roadway network variable: mpo
2020-10-16 11:53:09, INFO: Calculating MPO as roadway network variable: mpo
2020-10-16 11:53:09, INFO: Finished calculating MPO variable: mpo
2020-10-16 11:53:09, INFO: Finished calculating MPO variable: mpo
2020-10-16 11:53:09, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2020-10-16 11:53:09, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2020-10-16 11:53:09, INFO: MPO Variable 'assign_group' updated

## Apply Manual Python Changes

#### Change 0 
reapply OSM crosswalk for links which are improperly tagged as residential assigngrp =50 


In [13]:
subjects_df = m_net.links_df.loc[(m_net.links_df['roadway'] != 'residential') & (m_net.links_df['assign_group'] == 50)][['model_link_id']]
subjects_df.shape

(22099, 1)

In [14]:
osm_asgngrp_crosswalk_df = pd.read_csv(parameters.osm_assgngrp_dict)

join_df = pd.merge(
    m_net.links_df,
    osm_asgngrp_crosswalk_df.rename(
        columns={
            "assign_group": "assignment_group_osm",
            "roadway_class": "roadway_class_osm"
        }
    ),
    how="left",
    on="roadway",
)

def func(ag, rdwy, osm_val, cur_val):
    if ag == 50:
        if rdwy != 'residential':
            return osm_val
        return cur_val
    return cur_val

join_df['update_assign_group'] = np.vectorize(func)(
    join_df.assign_group, 
    join_df.roadway,
    join_df.assignment_group_osm,
    join_df.assign_group,
)

join_df['roadway_class'] = np.vectorize(func)(
    join_df.assign_group, 
    join_df.roadway,
    join_df.roadway_class_osm,
    join_df.roadway_class,
)

m_net.links_df['assign_group'] = join_df['update_assign_group']
m_net.links_df['roadway_class'] = join_df['roadway_class']

In [15]:
check_df = m_net.links_df.loc[m_net.links_df['model_link_id'].isin(subjects_df.model_link_id)]
check_df.assign_group.value_counts()

7     18789
6      3133
15       67
50       65
1        45
Name: assign_group, dtype: int64

#### Change 01
ROADWAY=motorway
in Areatypes 2, 3, 4, 5 -  assign_group= 1 (metered freeway)
In other areatype -  assign_group = 2 (unmetered freeway)

In [16]:
m_net.links_df.loc[(m_net.links_df['roadway'] == 'motorway') & (m_net.links_df['area_type'].isin([2,3,4,5])), 'assign_group'] = 1
m_net.links_df.loc[(m_net.links_df['roadway'] == 'motorway') & (~m_net.links_df['area_type'].isin([2,3,4,5])), 'assign_group'] = 2

#### Change 02
ROADWAY = trunk
Asgngrp=15 (expressway), overwriting any mrcc conflation 

In [17]:
m_net.links_df.loc[m_net.links_df['roadway'] == 'trunk', 'assign_group'] = 15

#### Change 03
Ramps - ROADWAY_CLASS=60 is supposed to equal Ramp, but finding some issues
 
       ROADWAY_CLASS= 60 & ROADWAY='motorway'
in Areatypes 2 3 4 5 - asgngrp 3 (metered ramp)
 
In other area types -  asgngrp =4 (unmetered ramp)

In [18]:
m_net.links_df.loc[(m_net.links_df['roadway_class'] == 60) & (m_net.links_df['roadway'] == 'motorway') & (m_net.links_df['area_type'].isin([2,3,4,5])), 'assign_group'] = 3
m_net.links_df.loc[(m_net.links_df['roadway_class'] == 60) & (m_net.links_df['roadway'] == 'motorway') & (~m_net.links_df['area_type'].isin([2,3,4,5])), 'assign_group'] = 4

#### Change 04
ROADWAY_CLASS= 60 & ROADWAY='trunk'
Appears to be slip ramps and acceleration lanes on expressways. Set as assign_group=4  (might need to revisit)

In [19]:
m_net.links_df.loc[(m_net.links_df['roadway_class'] == 60) & (m_net.links_df['roadway'] == 'trunk'), 'assign_group'] = 4

#### Change 05 NEW
changing collectors outside metro in MN to arterials for increased speed and volume 

In [20]:
m_net.links_df.loc[(m_net.links_df['county'].isin([11,12,13,14,15,18,19,20,22])) & (m_net.links_df['assign_group'] == 7), 'assign_group'] = 6

#### Change 06
Check for any links that are coded as assign_group = 3, 4 , 13 , 14 that don't attach to asgngrp= 1, 2, or 15 – ie ramps that don’t connect to freeway or expressway
These should be flagged for review

In [21]:
join_df = pd.merge(
    m_net.links_df.loc[m_net.links_df['assign_group'].isin([3, 4, 13, 14])][['A', 'B', 'assign_group']], 
    m_net.links_df[['A', 'B', 'assign_group']].rename(columns = {'assign_group': 'next_assign_group'}),
    how = 'left', 
    left_on = 'B',  
    right_on = 'A',
).drop(columns = ['A_y', 'B_y']).rename(columns = {'A_x': 'A', 'B_x': 'B'}).dropna().astype(int)

join_df = pd.merge(
    join_df,
    m_net.links_df[['A', 'B', 'assign_group']].rename(columns = {'assign_group': 'prev_assign_group'}),
    how = 'left',
    left_on = 'A',
    right_on = 'B',
).drop(columns = ['A_y', 'B_y']).rename(columns = {'A_x': 'A', 'B_x': 'B'}).dropna().astype(int)

def func(n, p):
    if np.in1d(n, [1, 2, 15]):
        return int(1)
    elif np.in1d(p, [1, 2, 15]):
        return int (1)
    return int(0)

join_df['ramp_flag'] = np.vectorize(func)(
    join_df.next_assign_group.astype(int),
    join_df.prev_assign_group.astype(int)
)
    
join_back_df = join_df.groupby(['A','B'])['ramp_flag'].sum().reset_index()
join_back_df['ramp_flag'] = np.where(join_back_df['ramp_flag'] > 0, 'Connected', 'Need to Examine')

m_net.links_df = pd.merge(
    m_net.links_df,
    join_back_df,
    how = 'left',
    on = ['A', 'B'])


#### Change 07
Arterials – we have most of the major roads tagged as Assign_Group=6 (undivided artertial)
 
Assign_group = 6 and there is no reverse value (no B-A) then set  Assign_group = 5 (divided arterial)
If assign_group = 6 and Assign_group.R=6, then keep assign_group=6 (undivided arterial)

In [22]:
reverse_df = m_net.links_df[['A', 'B', 'drive_access']].rename(columns = {'B': 'A', 'A': 'B'})
reverse_df['reverse'] = 1
reverse_df = reverse_df[reverse_df['drive_access'] == 1].drop(columns = ['drive_access'])

join_df = pd.merge(
    m_net.links_df.loc[m_net.links_df['assign_group'] == 6][['A', 'B', 'assign_group']],
    reverse_df,
    how = 'left',
    on = ['A', 'B'],
)

join_df['update_assign_group'] = np.where(join_df['reverse'] == 1.0, 6, 5)

m_net.links_df = pd.merge(
     m_net.links_df,
     join_df.drop(columns = ['reverse', 'assign_group']),
     how = 'left',
     on = ['A', 'B'],
)

m_net.links_df.loc[(m_net.links_df['assign_group'] == 6) & (m_net.links_df['update_assign_group'] == 5), 'assign_group'] = 5
m_net.links_df = m_net.links_df.drop(columns = ['update_assign_group'])

 # Lane Changes
 


#### lane change #1 - set all residential back to 1 lane 


In [23]:
m_net.links_df.loc[(m_net.links_df['assign_group'] == 50) & (m_net.links_df['area_type'] != 5), 'lanes'] = 1

## Write to Disk

In [ ]:
m_net.links_metcouncil_df = m_net.links_df
m_net.links_metcouncil_df = pd.merge(
    m_net.links_metcouncil_df.drop("geometry", axis = 1),  
    m_net.shapes_df[["shape_id", "geometry"]],
    how = "left",
    on = "shape_id"
)

### As Shapefile

In [ ]:
#didn't run this 
#m_net.links_metcouncil_df = m_net.links_df
#out_cols = ['A','B','model_link_id', 'shape_id', 'assign_group', 'drive_access', 'roadway_class', 
#            'lanes_AM', 'lanes_MD', 'lanes_PM', 'lanes_NT', 
 #           'roadway', 'ramp_flag', 'geometry']

In [ ]:
#option 1 - short variables didn't run this 
#m_net.write_roadway_as_shp(
 #   output_link_shp = os.path.join(output_dir, 'v01_links.shp'),
   # output_node_shp = os.path.join(output_dir, 'v01_nodes.shp'),
  #  link_output_variables = out_cols,
    #data_to_csv = False, 
    #data_to_dbf = True,
#)




In [ ]:
#option two, more variables 
m_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'v01_links06.shp'),
    output_node_shp = os.path.join(output_dir, 'v01_nodes06.shp'),
    link_output_variables = variables_in_shape_file,
    data_to_csv = False, 
    data_to_dbf = True,
)


### As Fixed width (for Cube)

In [24]:
m_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'v04_links.txt'),
    output_node_txt = os.path.join(output_dir, 'v04_nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
)

2020-10-16 12:03:00, INFO: Starting fixed width conversion
2020-10-16 12:03:00, INFO: Starting fixed width conversion
2020-10-16 12:22:16, INFO: Writing out link database
2020-10-16 12:22:16, INFO: Writing out link database
2020-10-16 12:22:29, INFO: Writing out link header and width ----
2020-10-16 12:22:29, INFO: Writing out link header and width ----
2020-10-16 12:22:29, INFO: Starting fixed width conversion
2020-10-16 12:22:29, INFO: Starting fixed width conversion
2020-10-16 12:25:03, INFO: Writing out node database
2020-10-16 12:25:03, INFO: Writing out node database
2020-10-16 12:25:05, INFO: Writing out node header and width
2020-10-16 12:25:05, INFO: Writing out node header and width


In [25]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_01_scenario.transit_net, parameters = parameters)
standard_transit_net.write_as_cube_lin(outpath = os.path.join(output_dir, "v01_transit.lin"))

2020-10-16 12:25:07, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso
2020-10-16 12:25:07, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso
2020-10-16 12:25:07, INFO: Converting GTFS Standard Properties to MetCouncil's Cube Standard
2020-10-16 12:25:07, INFO: Converting GTFS Standard Properties to MetCouncil's Cube Standard


### As Network Standard

In [ ]:
v_01_scenario.road_net.links_df = pd.merge(
            v_01_scenario.road_net.links_df,
            m_net.links_df[['model_link_id', 'assign_group']],
            how = 'left',
            on = 'model_link_id',
)

v_01_scenario.road_net.write(path = output_dir, filename = 'v01')

In [ ]:
v_01_scenario.road_net.links_df.columns

### Scenario as Pickle

In [26]:
scenario_filename = os.path.join(pickle_dir, 'working_scenario_01.pickle')
pickle.dump(v_01_scenario, open(scenario_filename, 'wb'))

In [27]:
os.chdir(output_dir)
p = Popen("make_cube.bat")
p.communicate()

(None, None)